Source: https://huggingface.co/TheBloke/Pygmalion-13B-SuperHOT-8K-GPTQ

In [5]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [7]:
!nvidia-smi

Sun Jul  2 14:42:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  Off |
|  0%   43C    P8    17W / 480W |      3MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
!python3 -c "import torch; print(torch.__version__)"

2.0.1+cu118


In [9]:
!conda list | sed -n "3p" && conda list | grep -i -E "cuda|torch"

# Name                    Version                   Build  Channel
cudatoolkit               11.8.0               h6a678d5_0  
torch                     2.0.1+cu118              pypi_0    pypi
torchaudio                2.0.2+cu118              pypi_0    pypi
torchvision               0.15.2+cu118             pypi_0    pypi


In [4]:
!pip install einops auto-gptq



  Using cached auto_gptq-0.2.2.tar.gz (52 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [106 lines of output]
      include dirs are: ['autogptq_cuda']
      running bdist_wheel
      /opt/conda/lib/python3.10/site-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
        warnings.warn(msg.format('we could not find ninja.'))
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/auto_gptq
      copying auto_gptq/__init__.py -> build/lib.linux-x86_64-cpython-310/auto_gptq
      creating build/lib.linux-x86_64-cpython-310/auto_gptq/eval_tasks
      copying auto_gptq/eval_tasks/_base.py -> build/lib.linux-x86_64-cpython-310/au

Failed to build auto-gptq
  Running setup.py install for auto-gptq ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for auto-gptq did not run successfully.
  │ exit code: 1
  ╰─> [110 lines of output]
      include dirs are: ['autogptq_cuda']
      running install
      /opt/conda/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/auto_gptq
      copying auto_gptq/__init__.py -> build/lib.linux-x86_64-cpython-310/auto_gptq
      creating build/lib.linux-x86_64-cpython-310/auto_gptq/eval_tasks
      copying auto_gptq/eval_tasks/_base.py -> build/lib.linux-x86_64-cpython-310/auto_gptq/eval_tasks
      copying auto_gptq/eval_tasks/language_modeling_tas

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/Pygmalion-13B-SuperHOT-8K-GPTQ"
model_basename = "pygmalion-13b-superhot-8k-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device_map='auto',
        use_triton=use_triton,
        quantize_config=None)

model.seqlen = 8192

# Note: check the prompt template is correct for this model.
prompt = "Tell me about AI"
prompt_template=f'''USER: {prompt}
ASSISTANT:'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print(pipe(prompt_template)[0]['generated_text'])